# Explore Features

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pandas.plotting import table
from scipy import stats

from PlantReactivityAnalysis.config import FEATURES_LETTERS_DIR, FIGURES_DIR
from PlantReactivityAnalysis.features.features_dataset import FeaturesDataset
from PlantReactivityAnalysis.visualization.visualize import export_df_to_image_formatted

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
def compute_stats_and_test(df, target_column):
    # Dynamically create column names based on unique classes for averages
    unique_classes = np.unique(df[target_column])
    avg_columns = [f'Class_{val}_Avg' for val in unique_classes]
    # Initialize an empty DataFrame to store the results with dynamic columns
    stats_df = pd.DataFrame(columns=['Feature'] + avg_columns + ['P_Value', 'Test_Used'])
    
    features = df.columns.drop(target_column)
    for feature in features:
        # Compute averages for each class
        averages = [df[df[target_column] == val][feature].mean() for val in unique_classes]
        
        # Prepare data for tests
        groups = [df[df[target_column] == val][feature].dropna() for val in unique_classes]
        
        # Check assumptions
        normality_pass = all(stats.shapiro(group)[1] > 0.05 for group in groups if len(group) > 3)  # Shapiro-Wilk test requires more than 3 values
        homogeneity_pass = stats.levene(*groups)[1] > 0.05 if len(groups) > 1 else True  # Levene's test requires at least two groups
        
        # Select and perform the appropriate test
        if len(groups) > 1:  # Only perform statistical tests if there are at least two groups to compare
            if normality_pass and homogeneity_pass:
                # Perform ANOVA
                test_statistic, p_value = stats.f_oneway(*groups)
                test_used = 'ANOVA'
            else:
                # Perform Kruskal-Wallis
                test_statistic, p_value = stats.kruskal(*groups)
                test_used = 'Kruskal-Wallis'
        else:
            p_value = np.nan
            test_used = 'Not applicable'
        
        # Append the results to the DataFrame
        row = [feature] + averages + [p_value, test_used]
        stats_df.loc[len(stats_df)] = row
    
    # Sort by P_Value while handling missing values (datasets with a single class)
    stats_df = stats_df.sort_values(by='P_Value', ascending=True, na_position='last')
    
    return stats_df

## Data Import

In [4]:
file_path = FEATURES_LETTERS_DIR / "features_dataset_norm_letters_ws1_hl0.5.pkl"
dataset= FeaturesDataset.load(file_path)

Dataset loaded from C:\Users\alvar\Documents\GitHub\Plant-Reactivity-Analysis\data\processed\segmented_by_letters\features_dataset_norm_letters_ws1_hl0.5.pkl. Shape: (8878, 187)


In [5]:
dataset.make_final_dataset()

Removing columns with NaN values: ['skewness', 'kurtosis']


In [6]:
dataset.features.shape

(8638, 179)

In [7]:
dataset.features.head()

,id_measurement,id_performance,datetime,plant,generation,num_eurythmy,eurythmy_letter,cepstra_1_avg,cepstra_2_avg,cepstra_3_avg,cepstra_4_avg,cepstra_5_avg,cepstra_6_avg,cepstra_7_avg,cepstra_8_avg,cepstra_9_avg,cepstra_10_avg,cepstra_11_avg,cepstra_12_avg,cepstra_13_avg,cepstra_1_std,cepstra_2_std,cepstra_3_std,cepstra_4_std,cepstra_5_std,cepstra_6_std,cepstra_7_std,cepstra_8_std,cepstra_9_std,cepstra_10_std,cepstra_11_std,cepstra_12_std,cepstra_13_std,zcr_mean,energy_mean,energy_entropy_mean,spectral_centroid_mean,spectral_spread_mean,spectral_entropy_mean,spectral_flux_mean,spectral_rolloff_mean,mfcc_1_mean,mfcc_2_mean,mfcc_3_mean,mfcc_4_mean,mfcc_5_mean,mfcc_6_mean,mfcc_7_mean,mfcc_8_mean,mfcc_9_mean,mfcc_10_mean,mfcc_11_mean,mfcc_12_mean,mfcc_13_mean,chroma_1_mean,chroma_2_mean,chroma_3_mean,chroma_4_mean,chroma_5_mean,chroma_6_mean,chroma_7_mean,chroma_8_mean,chroma_9_mean,chroma_10_mean,chroma_11_mean,chroma_12_mean,chroma_std_mean,delta zcr_mean,delta energy_mean,delta energy_entropy_mean,delta spectral_centroid_mean,delta spectral_spread_mean,delta spectral_entropy_mean,delta spectral_flux_mean,delta spectral_rolloff_mean,delta mfcc_1_mean,delta mfcc_2_mean,delta mfcc_3_mean,delta mfcc_4_mean,delta mfcc_5_mean,delta mfcc_6_mean,delta mfcc_7_mean,delta mfcc_8_mean,delta mfcc_9_mean,delta mfcc_10_mean,delta mfcc_11_mean,delta mfcc_12_mean,delta mfcc_13_mean,delta chroma_1_mean,delta chroma_2_mean,delta chroma_3_mean,delta chroma_4_mean,delta chroma_5_mean,delta chroma_6_mean,delta chroma_7_mean,delta chroma_8_mean,delta chroma_9_mean,delta chroma_10_mean,delta chroma_11_mean,delta chroma_12_mean,delta chroma_std_mean,zcr_std,energy_std,energy_entropy_std,spectral_centroid_std,spectral_spread_std,spectral_entropy_std,spectral_flux_std,spectral_rolloff_std,mfcc_1_std,mfcc_2_std,mfcc_3_std,mfcc_4_std,mfcc_5_std,mfcc_6_std,mfcc_7_std,mfcc_8_std,mfcc_9_std,mfcc_10_std,mfcc_11_std,mfcc_12_std,mfcc_13_std,chroma_1_std,chroma_2_std,chroma_3_std,chroma_4_std,chroma_5_std,chroma_6_std,chroma_7_std,chroma_8_std,chroma_9_std,chroma_10_std,chroma_11_std,chroma_12_std,chroma_std_std,delta zcr_std,delta energy_std,delta energy_entropy_std,delta spectral_centroid_std,delta spectral_spread_std,delta spectral_entropy_std,delta spectral_flux_std,delta spectral_rolloff_std,delta mfcc_1_std,delta mfcc_2_std,delta mfcc_3_std,delta mfcc_4_std,delta mfcc_5_std,delta mfcc_6_std,delta mfcc_7_std,delta mfcc_8_std,delta mfcc_9_std,delta mfcc_10_std,delta mfcc_11_std,delta mfcc_12_std,delta mfcc_13_std,delta chroma_1_std,delta chroma_2_std,delta chroma_3_std,delta chroma_4_std,delta chroma_5_std,delta chroma_6_std,delta chroma_7_std,delta chroma_8_std,delta chroma_9_std,delta chroma_10_std,delta chroma_11_std,delta chroma_12_std,delta chroma_std_std,zero_crossing_rate,root_mean_square_energy,slope_sign_changes_ratio,hjorth_mobility,hjorth_complexity,mean,variance,standard_deviation,interquartile_range,dfa
0,1,1,2023-04-29,salad,1,1,A1,0.389257,0.355223,0.315986,0.303726,0.293090,0.287535,0.273950,0.261706,0.257738,0.259543,0.254311,0.243789,0.235699,1.732068,1.131020,1.039882,0.974513,0.963384,0.934806,0.901450,0.919531,0.873621,0.818862,0.795026,0.768581,0.717852,0.000039,0.088304,3.261203,0.032223,0.081098,0.004826,0.018708,0.000211,-62.296443,1.235709,0.155466,0.211200,0.092647,0.102716,0.061738,0.059721,0.037312,0.030640,0.018006,0.026201,0.015822,0.000044,0.000008,0.001370,0.000043,0.196797,0.000024,0.005580,0.000003,0.000010,0.000034,0.000326,0.000012,0.054775,0.0,-0.000072,-0.001124,0.000538,0.001182,0.000063,2.189057e-04,0.0,0.393135,0.013044,0.001649,0.002161,0.000401,-0.000133,-0.000671,0.000761,0.001470,0.000749,-0.000250,0.000223,0.000054,8.808522e-07,1.245783e-07,0.000015,1.390759e-07,-0.000043,2.665300e-07,0.000083,6.089822e-08,2.136081e-07,6.515224e-07,2.399732e-06,1.328375e-07,-1.396324e-05,0.000155,0.053899,0.099150,0.019595,0.044021,0.010882,0.027826,0.000890,17.710120,0.626208,0.098801,0.128861,0.079956,0.096823,0.064831,0.082795

In [8]:
df= dataset.label_features

## Exploration

In [9]:
for i, column in enumerate(df.columns, start=1):
    print(f"{i}. {column}")

1. id_measurement
2. id_performance
3. datetime
4. plant
5. generation
6. num_eurythmy
7. eurythmy_letter


## RQ1: Eurythmy/Control

In [10]:
rq = dataset.return_subset_given_research_question(1)
train_cols, ttest = rq.reduce_features_based_on_target(corr_threshold=0.8)
df= rq.objective_features
df.describe()

Reduced variable features from initial count to 44.


,cepstra_4_avg,cepstra_1_std,cepstra_8_std,energy_mean,mfcc_9_mean,mfcc_11_mean,mfcc_12_mean,mfcc_13_mean,chroma_std_mean,delta zcr_mean,delta energy_mean,delta energy_entropy_mean,delta spectral_spread_mean,delta spectral_flux_mean,delta spectral_rolloff_mean,delta mfcc_2_mean,delta mfcc_3_mean,delta mfcc_5_mean,delta mfcc_6_mean,delta mfcc_7_mean,delta mfcc_8_mean,delta mfcc_9_mean,delta mfcc_10_mean,delta mfcc_11_mean,delta mfcc_12_mean,delta mfcc_13_mean,delta chroma_3_mean,delta chroma_4_mean,delta chroma_std_mean,energy_std,delta energy_std,delta spectral_centroid_std,delta spectral_spread_std,delta mfcc_12_std,delta chroma_8_std,root_mean_square_energy,slope_sign_changes_ratio,hjorth_mobility,hjorth_complexity,mean,variance,interquartile_range,dfa,target
count,8638.000000,8638.000000,8638.000000,8638.000000,8638.000000,8638.000000,8638.000000,8638.000000,8638.000000,8.638000e+03,8638.000000,8638.000000,8.638000e+03,8638.000000,8.638000e+03,8638.000000,8638.000000,8.638000e+03,8.638000e+03,8.638000e+03,8.638000e+03,8638.000000,8.638000e+03,8638.000000,8638.000000,8638.000000,8.638000e+03,8.638000e+03,8.638000e+03,8638.000000,8638.000000,8638.000000,8638.000000,8638.000000,8.638000e+03,8638.000000,8638.000000,8.638000e+03,8.638000e+03,8638.000000,8638.000000,8638.000000,8638.000000,8638.00000
mean,0.231959,1.434192,0.764238,0.142223,0.048298,0.036143,0.038542,0.027227,0.055073,-1.222496e-07,0.000102,-0.000004,-5.638984e-07,0.000117,5.290581e-08,-0.000015,0.000001,3.500990e-06,3.172874e-06,-1.321584e-05,-4.770658e-06,0.000003,9.827149e-06,0.000009,0.000005,-0.000001,1.113864e-06,2.483149e-08,1.602757e-07,0.082603,0.047114,0.037737,0.069874,0.119684,4.320477e-05,0.805437,0.019290,1.897173e-05,1.073388e+05,-0.018244,0.586970,0.759324,1.667287,0.49699
std,0.054420,0.165175,0.291974,0.089543,0.015170,0.012283,0.012281,0.010203,0.001354,1.633872e-05,0.001918,0.001779,5.489848e-04,0.000867,3.939303e-05,0.006784,0.001154,8.894905e-04,9.178815e-04,8.050021e-04,8.626725e-04,0.000864,8.499533e-04,0.000807,0.000746,0.000694,1.122345e-04,2.935303e-06,4.277880e-05,0.033434,0.026559,0.015117,0.025925,0.025451,3.492923e-05,0.509805,0.041758,2.922485e-05,9.342953e+04,0.566863,1.107337,0.606975,0.124139,0.50002
min,0.009944,0.590801,0.140652,0.002094,-0.004899,-0.007581,-0.003782,-0.004399,0.052889,-8.972490e-04,-0.006225,-0.019428,-1.919619e-03,-0.004247,-1.345455e-03,-0.028958,-0.010954,-5.533875e-03,-6.689900e-03,-6.450216e-03,-6.832760e-03,-0.008260,-6.850025e-03,-0.006727,-0.004956,-0.004803,-1.088257e-03,-3.058370e-05,-2.692723e-04,0.001722,0.002389,0.003753,0.014094,0.005947,4.143687e-07,0.029174,0.000218,9.278176e-07,1.016612e+03,-3.808845,0.000100,0.000000,0.764225,0.00000
25%,0.197538,1.336546,0.597539,0.078015,0.038480,0.028356,0.030303,0.020677,0.054749,0.000000e+00,-0.000772,-0.000060,-3.630463e-04,-0.000126,-1.124244e-20,-0.000731,-0.000581,-4.601364e-04,-4.858454e-04,-3.747912e-04,-3.959626e-04,-0.000315,-3.401780e-04,-0.000304,-0.000301,-0.000271,-9.850440e-06,-2.179706e-07,-1.755783e-06,0.057566,0.026966,0.026622,0.050191,0.105731,2.067888e-05,0.471070,0.004502,1.017558e-05,5.703358e+04,-0.344130,0.088234,0.313764,1.589192,0.00000
50%,0.232286,1.440971,0.731661,0.123342,0.050869,0.037886,0.040325,0.028057,0.055021,0.000000e+00,-0.000059,0.000033,-1.375512e-12,-0.000004,0.000000e+00,0.000102,0.000029,9.163763e-07,3.850384e-07,-2.817397e-07,-8.335121e-08,-0.000004,-5.399829e-07,-0.000015,-0.000010,-0.000023,-6.368817e-07,-1.289684e-08,2.387282e-07,0.078574,0.041561,0.036774,0.062503,0.121796,3.680635e-05,0.708769,0.007873,1.538935e-05,8.171272e+04,-0.024362,0.271246,0.621321,1.668530,0.00000
75%,0.266607,1.542298,0.869689,0.186190,0.059624,0.044970,0.047836,0.034038,0.055259,0.000000e+00,0.000562,0.000494,3.439316e-04,0.000117,0.000000e+00,0.004435,0.000657,5.027486e-04,5.050579e-04,3.655214e-04,3.869813e-04,0.000322,3.388676e-04,0.000299,0.000283,0.000251,7.080609e-07,1.370065e-08,4.533512e-06,0.103552,0.061728,0.047787,0.0

In [11]:
stats_df = compute_stats_and_test(df, 'target')
stats_df.columns= ['Feature', 'Control Avg', 'Eurythmy Avg', 'P-Value', 'Test Used']
stats_df = stats_df.sort_values(by='P-Value', ascending=True)
stats_df.head(30)

,Feature,Control Avg,Eurythmy Avg,P-Value,Test Used
36,slope_sign_changes_ratio,2.971990e-02,8.734502e-03,0.000000e+00,Kruskal-Wallis
4,mfcc_9_mean,5.344916e-02,4.308518e-02,1.807461e-239,Kruskal-Wallis
6,mfcc_12_mean,4.260532e-02,3.442938e-02,5.805849e-225,Kruskal-Wallis
7,mfcc_13_mean,3.033443e-02,2.408293e-02,2.285189e-207,Kruskal-Wallis
5,mfcc_11_mean,4.004473e-02,3.219476e-02,6.011778e-206,Kruskal-Wallis
32,delta spectral_spread_std,6.324601e-02,7.658217e-02,8.097108e-135,Kruskal-Wallis
42,dfa,1.696072e+00,1.638154e+00,1.886160e-123,Kruskal-Wallis
30,delta energy_std,5.086293e-02,4.332017e-02,1.658450e-35,Kruskal-Wallis
40,variance,6.926854e-01,4.799748e-01,8.633461e-33,Kruskal-Wallis
41,interquartile_range,8.214935e-01,6.964012e-01,6.493563e-27,Kruskal-Wallis


In [12]:
filtered_df = stats_df[stats_df['P-Value'] < 0.05]
col_widths = [0.17, 0.1, 0.1, 0.1, 0.1] 
export_df_to_image_formatted(filtered_df, FIGURES_DIR/'statistical_test_rq1.png', col_widths=col_widths)

DataFrame exported as image to C:\Users\alvar\Documents\GitHub\Plant-Reactivity-Analysis\reports\figures\statistical_test_rq1.png


## RQ 2: A/G/D

In [13]:
rq = dataset.return_subset_given_research_question(2)
train_cols, ttest = rq.reduce_features_based_on_target(corr_threshold=0.8)
df= rq.objective_features
df.describe()

Reduced variable features from initial count to 40.


,cepstra_13_avg,cepstra_1_std,cepstra_13_std,energy_mean,mfcc_8_mean,mfcc_12_mean,chroma_std_mean,delta zcr_mean,delta energy_mean,delta energy_entropy_mean,delta spectral_spread_mean,delta spectral_flux_mean,delta mfcc_3_mean,delta mfcc_5_mean,delta mfcc_6_mean,delta mfcc_7_mean,delta mfcc_8_mean,delta mfcc_9_mean,delta mfcc_10_mean,delta mfcc_11_mean,delta mfcc_12_mean,delta mfcc_13_mean,delta chroma_2_mean,delta chroma_std_mean,energy_std,delta energy_std,delta spectral_flux_std,delta mfcc_6_std,delta mfcc_10_std,delta mfcc_12_std,delta chroma_std_std,zero_crossing_rate,slope_sign_changes_ratio,hjorth_mobility,hjorth_complexity,mean,variance,interquartile_range,dfa,target
count,3616.000000,3616.000000,3616.000000,3616.000000,3616.000000,3616.000000,3616.000000,3.616000e+03,3616.000000,3616.000000,3.616000e+03,3616.000000,3616.000000,3.616000e+03,3.616000e+03,3.616000e+03,3.616000e+03,3616.000000,3616.000000,3616.000000,3616.000000,3.616000e+03,3.616000e+03,3.616000e+03,3616.000000,3616.000000,3616.000000,3616.000000,3616.000000,3616.000000,3616.000000,3616.000000,3616.000000,3616.000000,3616.000000,3616.000000,3616.000000,3616.000000,3616.000000,3616.000000
mean,0.174513,1.443657,0.630160,0.154024,0.071363,0.034435,0.054992,7.689739e-08,0.000099,-0.000027,2.452891e-07,0.000115,0.000007,-5.734676e-06,-8.651250e-06,-1.287735e-05,-1.092832e-06,0.000006,0.000018,0.000010,0.000008,7.529046e-07,1.009644e-08,-3.666785e-07,0.082351,0.043539,0.106524,0.150478,0.136578,0.120440,0.004313,0.000044,0.008743,0.000019,99439.087701,-0.002468,0.493052,0.708254,1.638426,0.996405
std,0.050189,0.156185,0.381466,0.095398,0.023187,0.012403,0.000483,4.114426e-06,0.002012,0.001717,5.857751e-04,0.000919,0.001236,9.363422e-04,9.721632e-04,8.176873e-04,9.088456e-04,0.000904,0.000876,0.000831,0.000774,6.945639e-04,7.238308e-07,3.622337e-05,0.033873,0.024205,0.073252,0.028612,0.031264,0.027578,0.002950,0.000049,0.018475,0.000013,72025.316223,0.629004,0.986088,0.592286,0.119902,0.818012
min,-0.075761,0.756275,0.081310,0.006364,0.002310,-0.002345,0.053028,-1.821825e-05,-0.005625,-0.017037,-1.919619e-03,-0.004247,-0.006916,-4.927854e-03,-6.689900e-03,-5.364207e-03,-6.832760e-03,-0.007249,-0.005703,-0.004398,-0.004956,-4.201057e-03,-1.183658e-05,-2.363723e-04,0.013444,0.002389,0.003358,0.016750,0.006030,0.005947,0.000113,0.000000,0.000218,0.000002,11604.964763,-3.808845,0.000100,0.000000,1.234314,0.000000
25%,0.144119,1.345983,0.422623,0.084550,0.055328,0.025697,0.054741,0.000000e+00,-0.000880,-0.000040,-4.007578e-04,-0.000123,-0.000721,-5.406208e-04,-5.671199e-04,-4.018771e-04,-4.203112e-04,-0.000307,-0.000359,-0.000292,-0.000279,-2.460028e-04,-3.680576e-08,-1.192823e-06,0.057790,0.025686,0.052424,0.134378,0.118586,0.105837,0.002105,0.000008,0.003565,0.000010,56996.716963,-0.386950,0.075421,0.274521,1.559054,0.000000
50%,0.177595,1.447930,0.541005,0.135107,0.073829,0.035275,0.055006,0.000000e+00,-0.000052,0.000020,-6.805454e-20,-0.000002,0.000001,3.407417e-20,6.623490e-19,-7.070560e-08,-4.250073e-19,-0.000009,-0.000005,-0.000009,-0.000008,-2.352699e-05,-1.957651e-09,1.467224e-07,0.077129,0.038159,0.088219,0.153352,0.139655,0.124101,0.003720,0.000030,0.005510,0.000015,82106.266751,-0.041074,0.225921,0.563464,1.636036,1.000000
75%,0.206973,1.551260,0.687633,0.201444,0.089441,0.043746,0.055223,0.000000e+00,0.000638,0.000336,3.781165e-04,0.000098,0.000770,5.615642e-04,5.530776e-04,3.684981e-04,4.067538e-04,0.000302,0.000335,0.000280,0.000272,2.272288e-04,2.942008e-09,3.173249e-06,0.102986,0.055830,0.141998,0.169386,0.157606,0.138776,0.005920,0.000064,0.008668,0.000022,118559.788605,0.352626,0.543399,0.985947,1.720460,2.000000
max,0.346045,2.197560,3.554670,0.720828,0.124238,0.070404,0.058679,5.010020e-05,0.007962,0.010430,2.087210e-03,0.007617,0.005040,4.775662e-03,4.833218e-03,5.314793e-03,7.078407e-03,0.004889,0.005138,0.004887,0.004684,4.167054e-03,2.375785e-05,4.998233e-04,0.245661,0.225082,0.447232,0.229727,0.237491,0.205196,0.019091,0.000423,0.362309,0.

In [14]:
stats_df = compute_stats_and_test(df, 'target')
stats_df = stats_df.sort_values(by='P_Value', ascending=True)
stats_df.columns= ['Feature', 'A Avg', 'G Avg', 'D Avg', 'P-Value', 'Test Used']
stats_df.head(9)

,Feature,A Avg,G Avg,D Avg,P-Value,Test Used
1,cepstra_1_std,1.425599,1.488751,1.417040,1.151958e-36,Kruskal-Wallis
0,cepstra_13_avg,0.170089,0.184540,0.169008,1.674200e-17,Kruskal-Wallis
35,mean,0.060413,-0.012824,-0.055726,6.290599e-05,Kruskal-Wallis
24,energy_std,0.080766,0.085103,0.081216,1.334879e-03,Kruskal-Wallis
34,hjorth_complexity,102281.986670,95432.981245,100551.593337,3.424500e-03,Kruskal-Wallis
33,hjorth_mobility,0.000018,0.000020,0.000019,4.754483e-03,Kruskal-Wallis
8,delta energy_mean,0.000126,-0.000021,0.000192,7.168732e-03,Kruskal-Wallis
25,delta energy_std,0.042461,0.045158,0.043020,1.395831e-02,Kruskal-Wallis
17,delta mfcc_9_mean,0.000017,-0.000040,0.000041,2.420865e-02,Kruskal-Wallis


In [15]:
filtered_df = stats_df[stats_df['P-Value'] < 0.05]
col_widths = [0.15, 0.1, 0.1, 0.1, 0.1, 0.1] 
export_df_to_image_formatted(filtered_df, FIGURES_DIR/'statistical_test_rq2.png', col_widths=col_widths)

DataFrame exported as image to C:\Users\alvar\Documents\GitHub\Plant-Reactivity-Analysis\reports\figures\statistical_test_rq2.png
